fair_facts_v2:

The Framework for Assessing Changes To Sea-level (FACTS) is an open-source modular, scalable, and extensive framework for global mean, regional, and extreme sea level projection that is designed to support the characterization of ambiguity in sea-level projections. It is designed so users can easily explore deep uncertainty by investigating the implications on GMSL, RSL, and ESL of different choices for different processes. Its modularity allows components to be represented by either simple or complex model. Because it is built upon the Radical-PILOT computing stack, different modules can be dispatched for execution on resources appropriate to their computational complexity.

FACTS is being developed by the Earth System Science & Policy Lab and the RADICAL Research Group at Rutgers University. FACTS is released under the MIT License.

Project Github Page: https://github.com/radical-collaboration/facts

In [1]:
!pip install radical-asyncflow

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import asyncio
import logging
import time
import os
import shlex
import random

from radical.asyncflow import WorkflowEngine
from radical.asyncflow import ConcurrentExecutionBackend

from concurrent.futures import ThreadPoolExecutor

from radical.asyncflow.logging import init_default_logger

logger = logging.getLogger(__name__)

In [3]:
async def modules():
    init_default_logger(logging.DEBUG)

    # Create backend and workflow
    engine = await ConcurrentExecutionBackend(ThreadPoolExecutor())
    flow = await WorkflowEngine.create(engine)
    
    # Ensure output directories exist
    def setup_directories():
        os.makedirs('./data/output/fair', exist_ok=True)
        os.makedirs('./data/output/lws', exist_ok=True)
        os.makedirs('./data/output/sterodynamics', exist_ok=True)

    @flow.executable_task
    async def fair_task():
        """FAIR temperature model task"""
        cmd = [
            '/usr/local/other/singularity/4.0.3/bin/singularity', 'exec',
            '--bind', '/discover/nobackup/projects/sealevel/facts2.0/data/input:/input',
            '--bind', './data/output/fair:/output',
            '/discover/nobackup/projects/sealevel/facts2.0//containers/fair-temperature.sif',
            'fair-temperature',
            '--pipeline-id=1234',
            '--output-oceantemp-file=/output/oceantemp.nc',
            '--nsamps=20',
            '--output-ohc-file=/output/ohc.nc',
            '--output-gsat-file=/output/gsat.nc',
            '--output-climate-file=/output/climate.nc',
            '--rcmip-file=/input/rcmip/rcmip-emissions-annual-means-v5-1-0.csv',
            '--param-file=/input/parameters/fair_ar6_climate_params_v4.0.nc'
        ]
        return shlex.join(cmd)

    @flow.executable_task
    async def lws_task():
        """Land Water Storage task - can run independently of FAIR"""
        cmd = [
            '/usr/local/other/singularity/4.0.3/bin/singularity', 'exec',
            '--bind', '/discover/nobackup/projects/sealevel/facts2.0/data/input:/input',
            '--bind', './data/output/lws:/output',
            '/discover/nobackup/projects/sealevel/facts2.0/containers/ssp-landwaterstorage.sif',
            'ssp-landwaterstorage',
            '--pipeline-id=1234',
            '--nsamps=20',
            '--output-gslr-file=/output/gslr.nc',
            '--output-lslr-file=/output/lslr.nc',
            '--location-file=/input/location.lst',
            '--pophist-file=/input/UNWPP2012 population historical.csv',
            '--reservoir-file=/input/Chao2008 groundwater impoundment.csv',
            '--popscen-file=/input/ssp_iam_baseline_popscenarios2100.csv',
            '--gwd-file=/input/Konikow2011 GWD.csv',
            '--gwd-file=/input/Wada2012 GWD.csv',
            '--gwd-file=/input/Pokhrel2012 GWD.csv',
            '--fp-file=/input/REL_GROUNDWATER_NOMASK.nc'
        ]
        return shlex.join(cmd)

    @flow.executable_task
    async def sterodynamics_task(fair_task):
        """Sterodynamics task - depends on FAIR output"""
        cmd = [
            '/usr/local/other/singularity/4.0.3/bin/singularity', 'exec',
            '--bind', '/discover/nobackup/projects/sealevel/facts2.0/data/input:/input',
            '--bind', './data/output/fair:/fair',
            '--bind', './data/output/sterodynamics:/output',
            '--nv',
            '/discover/nobackup/projects/sealevel/facts2.0/containers/tlm-sterodynamics.sif',
            'tlm-sterodynamics',
            '--pipeline-id=1234',
            '--scenario=ssp585',
            '--nsamps=20',
            '--model-dir=/input/cmip6/',
            '--location-file=/input/location.lst',
            '--output-lslr-file=/output/lslr.nc',
            '--output-gslr-file=/output/gslr.nc',
            '--expansion-coefficients-file=/input/scmpy2LM_RCMIP_CMIP6calpm_n18_expcoefs.nc',
            '--gsat-rmses-file=/input/scmpy2LM_RCMIP_CMIP6calpm_n17_gsat_rmse.nc',
            '--climate-data-file=/fair/climate.nc'
        ]
        return shlex.join(cmd)

    async def run_climate_workflow(pipeline_id):
        """Run the complete climate workflow"""
        logger.info(f'Starting climate workflow {pipeline_id} at {time.time()}')

        # Setup directories
        setup_directories()

        # Start FAIR and LWS tasks (they can run in parallel)
        fair_future = fair_task()
        lws_future = lws_task()

        # Wait for FAIR to complete (sterodynamics depends on it)
        fair_result = await fair_future
        logger.info(f'FAIR task completed for pipeline {pipeline_id}')

        # Start sterodynamics task (depends on FAIR output)
        sterodynamics_future = sterodynamics_task(fair_future)

        # Wait for all tasks to complete
        lws_result = await lws_future
        sterodynamics_result = await sterodynamics_future

        logger.info(f'Climate workflow {pipeline_id} finished at {time.time()}')

        return {
            'fair': fair_result,
            'lws': lws_result,
            'sterodynamics': sterodynamics_result
        }

    # Run workflow(s)
    transationId = random.randint(1, 1000)
    logger.info("Launching asynchronous workflow: "+str(transationId))
    results = await run_climate_workflow(transationId)
    logger.info("All modules completed successfully: "+str(transationId))
    
    await flow.shutdown()
    logger.info(results)
    logger.info("All workflows completed successfully")

In [4]:
import asyncio
import subprocess

from pathlib import Path

def file_exists_and_has_content(filepath):
    path = Path(filepath)
    return path.is_file() and path.stat().st_size > 0
    
async def total():
    init_default_logger(logging.DEBUG)

    # Create backend and workflow
    engine = await ConcurrentExecutionBackend(ThreadPoolExecutor())
    flow = await WorkflowEngine.create(engine)
    
    # Ensure output directories exist
    def setup_total_directories():
        os.makedirs('./data/output', exist_ok=True)

    @flow.executable_task
    async def total_task(component, name):
        """Facts total task - executes singularity command"""
        filename = ""
        if (component == 'all'):
            filename = '/mnt/total_out/totaled_output_all_'+name+'.nc'
            _filename = './data/output/totaled_output_all_'+name+'.nc'
            cmd = [
                '/usr/local/other/singularity/4.0.3/bin/singularity', 'exec',
                '--bind', './data/output:/mnt/total_in',
                '--bind', './data/output:/mnt/total_out',
                '/discover/nobackup/projects/sealevel/facts2.0/containers/sealevel-facts-total_latest-sandbox',
                'facts-total',
                '--item=/mnt/total_out/lws/'+name+'.nc',
                '--item=/mnt/total_out/sterodynamics/'+name+'.nc',
                '--pyear-start=2020',
                '--pyear-end=2150',
                '--pyear-step=10',
                '--output-path='+filename
            ]
        else:
            filename = '/mnt/total_out/totaled_output_'+component+'_'+name+'.nc'
            _filename = './data/output/totaled_output_'+component+'_'+name+'.nc'
            cmd = [
                '/usr/local/other/singularity/4.0.3/bin/singularity', 'exec',
                '--bind', './data/output:/mnt/total_in',
                '--bind', './data/output:/mnt/total_out',
                '/discover/nobackup/projects/sealevel/facts2.0/containers/sealevel-facts-total_latest-sandbox',
                'facts-total',
                '--item=/mnt/total_out/'+component+'/'+name+'.nc',
                '--pyear-start=2020',
                '--pyear-end=2150',
                '--pyear-step=10',
                '--output-path='+filename
            ]
        
        # Log the command
        cmd_str = shlex.join(cmd)
        logger.info(f"Executing: {cmd_str}")
        
        # RUN THE COMMAND ASYNCHRONOUSLY
        proc = await asyncio.create_subprocess_exec(
            *cmd,
            stdout=asyncio.subprocess.PIPE,
            stderr=asyncio.subprocess.PIPE
        )
        
        stdout, stderr = await proc.communicate()

        # Verify that file was created
        path = Path(_filename)
        if path.is_file() and path.stat().st_size > 0:
            logger.info(f"Output successfully created: {path}")
        else:            
            error_msg = f"Output unsuccessfully created: {path}"
            logger.error(f"Command failed with return code: {error_msg}")
            raise RuntimeError(f"Task failed: {error_msg}")
        
        logger.info(f"Command output: {stdout.decode()}")
        if stderr:
            logger.warning(f"Command stderr: {stderr.decode()}")
            
        return {
            'command': cmd_str,
            'component': component,
            'name': name,
            'returncode': proc.returncode
        }

    async def run_total_workflow(pipeline_id):
        """Run the total climate workflow"""
        logger.info(f'Starting total climate workflow {pipeline_id} at {time.time()}')

        # Setup directories
        setup_total_directories()
        
        # Start ALL tasks in parallel (don't await yet)
        total_future_lws_lslr = total_task('lws','lslr')
        total_future_lws_gslr = total_task('lws','gslr')
        total_future_sterodynamics_lslr = total_task('sterodynamics','lslr')
        total_future_sterodynamics_gslr = total_task('sterodynamics','gslr')
        total_future_all_lslr = total_task('all','lslr')
        total_future_all_gslr = total_task('all','gslr')

        results = None
        try:
            results = await asyncio.wait_for(
                asyncio.gather(
                    total_future_lws_lslr,
                    total_future_lws_gslr,
                    total_future_sterodynamics_lslr,
                    total_future_sterodynamics_gslr,
                    total_future_all_lslr,
                    total_future_all_gslr,
                    return_exceptions=True
                ),
                timeout=60  # 1 minute timeout
                # timeout=300  # 5 minute timeout
            )
            return results
        except asyncio.TimeoutError:
            logger.info("Tasks terminated after 1 minutes, but all outputs are available")
            print(results)

        logger.info(f'ALL TOTAL tasks completed for pipeline {pipeline_id}')
        logger.info(f'Climate workflow {pipeline_id} finished at {time.time()}')
        return results
        
    # Run workflow(s)
    transationId = random.randint(1, 1000)
    results = await run_total_workflow(transationId)
    logger.info(results)
    logger.info("=========Total completed successfully=========: "+ str(transationId))
    await flow.shutdown()

In [5]:
# Just call it with await in Jupyter
try:
    await modules()
    await total()    
except Exception as e:
    print(f"Unexpected error, RESTART kernel and re-run Notebook: {e}")
    print(f"Notebook will NOT overwrite files, so manually clean up previous runs or change directory paths if a write error occurs")

2026-02-13 15:24:20.332 │ INFO │ [root] │ Logger configured successfully - Console: DEBUG, File: disabled (N/A), Structured: disabled, Style: modern
2026-02-13 15:24:20.332 │ INFO │ [execution.backend(concurrent)] │ ThreadPoolExecutor execution backend started successfully
2026-02-13 15:24:20.332 │ DEBUG │ [workflow_manager] │ Registered signal handler for SIGHUP
2026-02-13 15:24:20.333 │ DEBUG │ [workflow_manager] │ Registered signal handler for SIGTERM
2026-02-13 15:24:20.334 │ DEBUG │ [workflow_manager] │ Registered signal handler for SIGINT
2026-02-13 15:24:20.334 │ DEBUG │ [workflow_manager] │ Started run component
2026-02-13 15:24:20.335 │ INFO │ [main] │ Launching asynchronous workflow: 459
2026-02-13 15:24:20.335 │ INFO │ [main] │ Starting climate workflow 459 at 1771014260.3352752
2026-02-13 15:24:20.338 │ DEBUG │ [workflow_manager] │ Registered task: 'fair_task' with id of task.000001
2026-02-13 15:24:20.338 │ DEBUG │ [workflow_manager] │ Registered task: 'lws_task' with id o